In [56]:
import torch
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import matplotlib.pyplot as plt
import numpy as np

In [57]:
#define the transformations that we want to apply
transformations = {'train' : transforms.Compose([transforms.Resize((32, 32)),
                                                 transforms.Grayscale(),
                                                 transforms.ToTensor()])}

In [58]:
#define the path to our datasets
train_path = "D:/Drive-F/Learn python/Notebooks/Notebooks/Mynotebooks/CAT AND DOG/cat_dog/cat_dogs_small/train"
valid_path = "D:/Drive-F/Learn python/Notebooks/Notebooks/Mynotebooks/CAT AND DOG/cat_dog/cat_dogs_small/validation"
dataset = {'train' : datasets.ImageFolder(train_path, transform = transformations['train']),
          'valid' : datasets.ImageFolder(valid_path, transform = transformations['train'])}

In [59]:
#load the dataset
data_loader = {'train' : DataLoader(dataset['train'], batch_size = 32, shuffle = True),
              'valid' : DataLoader(dataset['valid'], batch_size = 32, shuffle = False)}

In [62]:
images, labels = next(iter(data_loader['train']))

In [63]:
images.shape

torch.Size([32, 1, 32, 32])

In [64]:
#see if cuda is available
device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [65]:
class LSTM(nn.Module):
    def __init__(self, input_dim, output_dim, hidden_dim, n_layers):
        super(LSTM, self).__init__()
        
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers = n_layers, batch_first = True)
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, x):
        #initialize the hidden state and the cell state with zeros
        h0 = torch.zeros(self.n_layers, x.size(0), self.hidden_dim).to(device)
        c0 = torch.zeros(self.n_layers, x.size(0), self.hidden_dim).to(device)
        
        out, (hidden, cell) = self.lstm(x, (h0, c0))
        
        output_ = out[:, -1, :]
        
        output = self.fc(output_)
        
        return output

In [66]:
#instatiate the model
SEQ_LEN = 32
INPUT_DIM = 32
OUTPUT_DIM = 2
HIDDEN_DIM = 128
N_LAYERS = 2
model = LSTM(input_dim = INPUT_DIM, output_dim = OUTPUT_DIM, hidden_dim = HIDDEN_DIM, n_layers = N_LAYERS)
model = model.to(device)
print(model)

LSTM(
  (lstm): LSTM(32, 128, num_layers=2, batch_first=True)
  (fc): Linear(in_features=128, out_features=2, bias=True)
)


In [67]:
#define the optimizer
optimizer = optim.Adam(model.parameters(), lr = 0.001)

In [68]:
#define the loss function
criterion = nn.CrossEntropyLoss()

In [91]:
#now we will TRAIN the model
def train_model(model, criterion, optimizer, num_epochs):
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-' * 10)

        for phase in ['train', 'valid']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0.0

            for input_, label in data_loader[phase]:
                inputs = input_.squeeze(1).to(device)
                labels = label.to(device)

                outputs = model(inputs)
                loss = criterion(outputs, labels)

                if phase == 'train':
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()

                _, preds = torch.max(outputs, 1)
                running_loss += loss.detach() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataset[phase])
            epoch_acc = running_corrects.float() / len(dataset[phase])

            print('{} loss: {:.4f}, acc: {:.4f}'.format(phase, epoch_loss.item(), epoch_acc.item()))
    return model

In [92]:
training = train_model(model = model, criterion = criterion, optimizer = optimizer, num_epochs = 10)

Epoch 1/10
----------
train loss: 0.6838, acc: 0.5545
valid loss: 0.6735, acc: 0.6080
Epoch 2/10
----------
train loss: 0.6720, acc: 0.5935
valid loss: 0.6641, acc: 0.6030
Epoch 3/10
----------
train loss: 0.6654, acc: 0.6040
valid loss: 0.6696, acc: 0.5960
Epoch 4/10
----------
train loss: 0.6652, acc: 0.5975
valid loss: 0.6649, acc: 0.6060
Epoch 5/10
----------
train loss: 0.6530, acc: 0.6215
valid loss: 0.6581, acc: 0.6180
Epoch 6/10
----------
train loss: 0.6507, acc: 0.6210
valid loss: 0.6661, acc: 0.6020
Epoch 7/10
----------
train loss: 0.6483, acc: 0.6235
valid loss: 0.6558, acc: 0.6150
Epoch 8/10
----------
train loss: 0.6427, acc: 0.6200
valid loss: 0.6617, acc: 0.6250
Epoch 9/10
----------
train loss: 0.6356, acc: 0.6290
valid loss: 0.6594, acc: 0.6140
Epoch 10/10
----------
train loss: 0.6354, acc: 0.6415
valid loss: 0.6567, acc: 0.6270
